Imports


In [51]:
!pip install antropy

import torch

import mne

import antropy


from sklearn.preprocessing import label_binarize

mne.set_log_level('ERROR')

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
import pandas as pd

import numpy as np
from scipy.fftpack import fft, ifft
from scipy.stats import entropy
from sklearn.preprocessing import LabelEncoder
from scipy.stats import skew
from scipy.stats import kurtosis
#NOTE USE LABEL ENCODER BEFORE RUNNING THE FULL VERSION 

Dataset and preprocessing


In [79]:
#this is where we actually extract the features 
def get_variability_measures(eeg_data):
    variability_features=[]
    std_value=np.std(eeg_data,axis=1)
    iqr_value = np.subtract(*np.percentile(eeg_data, [75, 25], axis=1))  # IQR of all channels
    variability_features.extend((np.mean(std_value),np.mean(iqr_value)))
    return variability_features

def get_distribution_features(eeg_data):
    distribution_features=[]
    skewness_value = skew(eeg_data, axis=1)
    kurtosis_value = kurtosis(eeg_data, axis=1) 
    distribution_features.extend((np.mean(skewness_value),np.mean(kurtosis_value)))
    return distribution_features
  
def zero_crossings(signal):
    # If signal crosses zero line, we'll have a change in sign of adjacent values
    return ((signal[:-1] * signal[1:]) < 0).sum()

def frequency_content_features(eeg_data):
    # Calculate zero crossings for each channel
    zero_crossings_values = np.apply_along_axis(zero_crossings, 1, eeg_data)
    
    mean_zero_crossings = np.mean(zero_crossings_values)
    return mean_zero_crossings

def hjorth_mobility(signal):
    first_derivative = np.diff(signal)
    variance = np.var(signal)
    variance_derivative = np.var(first_derivative)
    mobility = np.sqrt(variance_derivative / variance)
    return mobility

def get_hjorth_parameters(eeg_data):
    hjorth_parameters=[]
    signal=eeg_data
    
    
    
    #hjorth complexity
    first_derivative = np.diff(signal)
    second_derivative = np.diff(first_derivative)
    mobility = hjorth_mobility(signal)
    mobility_derivative = hjorth_mobility(first_derivative)
    complexity = mobility_derivative / mobility
    hjorth_parameters.extend((mobility,complexity))
    return hjorth_parameters

def get_higuchi_fractal_dimension(eeg_data):
    signal=eeg_data
    kmax=6
    Lk = []
    for k in range(1, kmax):
        Lm = []
        for m in range(0, k):
            Lmk = 0
            for i in range(1, int(np.floor((len(signal) - m) / k))):
                Lmk += abs(signal[m + i * k] - signal[m + i * k - k])
            Lmk = Lmk * (len(signal) / (k * int(np.floor((len(signal) - m) / k))))
            Lm.append(Lmk)
        Lk.append(np.log(np.mean(Lm)))
    (slope, _) = np.polyfit(np.log(range(1, kmax)), Lk, 1)
    return -slope

def get_entropies(eeg_data):
    entropies=[]
    signal=eeg_data
    #shanon entropy
    hist, _ = np.histogram(signal, bins=64, density=True)
    shannon_entropy=entropy(hist)
    #spectral entropy
    spectral_entropy=antropy.spectral_entropy(eeg_data,200,'welch',nperseg=None,normalize=True)
    #binned_entorpy
    hist, _ = np.histogram(signal, bins=10)
    binned_entropy=entropy(hist)
    entropies.extend((shannon_entropy,spectral_entropy,binned_entropy))
    return entropies
    
def get_power_bands(new_raw,tmin,tmax):
    
    power_features=[]
    

# Define frequency bands
    bands = {
        'delta': (0.5, 4),
        'theta': (4, 8),
        'alpha': (8, 13),
        'beta': (13, 30),
        'gamma': (30, None)  # Assuming Gamma is 30+ Hz
    }

# Dictionary to hold power values for each band
    band_power = {}

    for band, (l_freq, h_freq) in bands.items():
        # Filter the data for each frequency band
        band_data = new_raw.copy().filter(l_freq=l_freq, h_freq=h_freq, picks='eeg', verbose=False)

        # Compute the PSD for the filtered data using the Welch method
        sfreq = raw.info['sfreq'] 
        
        spectrum = band_data.compute_psd(method='welch', fmin=l_freq, fmax=np.inf,picks='eeg',verbose=False)

        psd=spectrum.get_data()
        # Integrate the PSD over the frequencies of interest to get the absolute power for each band
        band_power[band] = psd.mean(axis=1).sum()



    
    power_features.extend(band_power.values())
    
    epsilon = 1e-6

    # Calculate power ratios using values from the dictionary
    delta_theta_ratio = band_power['delta'] / (band_power['theta'] + epsilon)
    delta_alpha_ratio = band_power['delta'] / (band_power['alpha'] + epsilon)
    theta_beta_ratio = band_power['theta'] / (band_power['beta'] + epsilon)
    alpha_gamma_ratio = band_power['alpha'] / (band_power['gamma'] + epsilon)
    beta_gamma_ratio = band_power['beta'] / (band_power['gamma'] + epsilon)

    # Append the calculated ratios 
    power_features.extend([delta_theta_ratio, delta_alpha_ratio, theta_beta_ratio, alpha_gamma_ratio, beta_gamma_ratio])

    # Calculate power sums using values from the dictionary
    delta_theta_sum = band_power['delta'] + band_power['theta']
    alpha_beta_sum = band_power['alpha'] + band_power['beta']
    theta_gamma_sum = band_power['theta'] + band_power['gamma']

    # Append the calculated sums 
    power_features.extend([delta_theta_sum, alpha_beta_sum, theta_gamma_sum])
    total_power=band_power['delta'] + band_power['theta']+ band_power['gamma']+band_power['alpha']
    power_features.append(total_power)
    return power_features
    

    

    

In [80]:

    
def extract_features(surrogate_segment, segment_info,tmin,tmax):
    features = []
    eeg_data = np.real(surrogate_segment)
    new_raw = mne.io.RawArray(eeg_data, segment_info)
    
    
    features.extend(get_variability_measures(eeg_data))
    features.extend(get_distribution_features(eeg_data))
    
    
    zero_crossing_feature = [frequency_content_features(eeg_data)]
    features.extend(zero_crossing_feature)
    
    # Extend with placeholders for missing features (Hjorth Mobility, etc.)
    features.extend(get_hjorth_parameters(eeg_data)) 
    
    higuchi_fractal_dimension=[get_higuchi_fractal_dimension(eeg_data)]
    features.extend(higuchi_fractal_dimension)
    
    features.extend(get_entropies(eeg_data))
    power_band_features = get_power_bands(new_raw,tmin,tmax)
    features.extend(power_band_features)
    
    
    
    
    return features


def create_raw_from_parquet(parquet_file):
    df=pd.read_parquet(parquet_file)
    data=df.to_numpy().T
    info=mne.create_info(ch_names=list(df.columns),sfreq=200,ch_types='eeg')
    raw=mne.io.RawArray(data,info)
    return raw

def get_duration(raw):
    num_samples=len(raw.times)
    sampling_freq=raw.info['sfreq']
    duration=np.floor(num_samples/sampling_freq)
    
    return duration

In [81]:



# Initialize an empty DataFrame for the new training data
columns = [
    'eeg_id', 
    'eeg_sub_id', 
    'patient_id', 
    'Standard Deviation (STD)', 
    'Inter-Quartile Range (IQR)', 
    'Skewness', 
    'Kurtosis', 
    'Number of Zero Crossings', 
    'Hjorth Mobility', 
    'Hjorth Complexity', 
    'Higuchi Fractal Dimension', 
    'Shannon Entropy', 
    'Spectral Entropy', 
    'Binned Entropy', 
    'Delta Power', 
    'Theta Power', 
    'Alpha Power', 
    'Beta Power', 
    'Gamma Power', 
    'Delta/Theta Ratio', 
    'Delta/Alpha Ratio', 
    'Theta/Beta Ratio', 
    'Alpha/Gamma Ratio', 
    'Beta/Gamma Ratio', 
    'Delta+Theta Power', 
    'Alpha+Beta Power', 
    'Theta+Gamma Power', 
    'Total Power',
    'expert_consensus'
]

new_train_df = pd.DataFrame(columns=columns)
#print(new_train_df.shape[1])

# Load the original training CSV
train_csv = pd.read_csv('/kaggle/input/hms-harmful-brain-activity-classification/train.csv')
count=0
# Loop through each row in the train CSV
for index, row in train_csv.iterrows():
    eeg_id = row['eeg_id']
    sub_id = row['eeg_sub_id']
    patient_id = row['patient_id']
    seizure_label = row['expert_consensus']
   
    # Load the corresponding parquet file as an mne Raw object
    raw = create_raw_from_parquet(f'/kaggle/input/hms-harmful-brain-activity-classification/train_eegs/{eeg_id}.parquet')
    
    sub_id_start_time=sub_id
    tmin = sub_id_start_time * 50
    tmax = tmin + 50  # This ensures a 50-second window

    # Adjust tmax to not exceed the recording
    tmax = min(tmax, raw.times[-1])
    
    # Additionally, ensure tmin does not exceed the adjusted tmax or the recording
    tmin = min(tmin, tmax - 0.001)
    segment = raw.crop(tmin, tmax)  # Adjust 'tmin' and 'tmax' as necessary
    segment_info=raw.info
    # Generate FT Surrogate for the segment
    ft_segment = fft(segment.get_data())
    random_phases = np.exp(2j * np.pi * np.random.rand(*ft_segment.shape))
    surrogate_data = np.abs(ft_segment) * random_phases
    surrogate_segment = ifft(surrogate_data)
    
    # Extract features from the surrogate segment
    features = extract_features(surrogate_segment,segment_info,tmin,tmax)
    #print(len(features))
    # Append to the new DataFrame
    new_row = [eeg_id, sub_id, patient_id] + features + [seizure_label]
    #print(len(new_row))
    new_train_df.loc[len(new_train_df)] = new_row
    ''''count+=1
    print(count)
    if count==1:
        break'''
# Save the new DataFrame to CSV
new_train_df.to_csv('feature_extracted_data.csv', index=False)


/opt/conda/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3787: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/opt/conda/lib/python3.10/site-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/opt/conda/lib/python3.10/site-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/opt/conda/lib/python3.10/site-packages/scipy/signal/_spectral_py.py:589: UserWarning: nperseg = 256 is greater than input length  = 1, using nperseg = 1
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,
/opt/conda/lib/python3.10/site-packages/antropy/entropy.py:253: RuntimeWarning: invalid value encountered in divide
  psd_norm = psd / psd.sum(axis=axis, keepdims=True)
/opt/conda/lib/python3.10/site-packages/antropy/entropy.py:256: RuntimeWarn

ValueError: No frequencies found between fmin=0.5 and fmax=inf

In [71]:
print(new_train_df)

       eeg_id  eeg_sub_id  patient_id  Standard Deviation (STD)  \
0  1628180742           0       42516                 30.275588   

   Inter-Quartile Range (IQR)  Skewness  Kurtosis  Number of Zero Crossings  \
0                   40.444734  0.051111 -0.078174                    291.65   

   Hjorth Mobility  Hjorth Complexity  ...  Delta/Theta Ratio  \
0         0.223952           4.905116  ...           0.691598   

   Delta/Alpha Ratio Theta/Beta Ratio  Alpha/Gamma Ratio  Beta/Gamma Ratio  \
0           0.729855         0.540029           1.112028          2.173113   

   Delta+Theta Power  Alpha+Beta Power  Theta+Gamma Power  Total Power  \
0         116.075163        192.086658         127.089996   239.568331   

   expert_consensus  
0           Seizure  

[1 rows x 29 columns]


In [72]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('/kaggle/input/hms-harmful-brain-activity-classification/train.csv')

# Calculate the percentage of each label
label_percentage = df['expert_consensus'].value_counts(normalize=True) * 100

print(label_percentage)


expert_consensus
Seizure    19.600187
GRDA       17.660112
Other      17.610487
GPD        15.638577
LRDA       15.580524
LPD        13.910112
Name: proportion, dtype: float64


Model Architure
